### Colorectal Cancer Dataset

In [1]:
# load modules
import pandas as pd
from src import ai_module as ai

In [2]:
# define data folder path and data file names

raw_data_folder = r"C:\Users\35799\Desktop\cookiecutter-analytical-project\biolizard-internship-marios\data\raw"

gene_expression_data = r"Colorectal Cancer Gene Expression Data.csv"
patient_data = r"Colorectal Cancer Patient Data.csv"

In [3]:
# load data

gene_expression_df = ai.data_load(raw_data_folder, gene_expression_data)
patient_df = ai.data_load(raw_data_folder, patient_data)

In [4]:
# combine data into one Pandas DataFrame

gene_expression_df_transposed = gene_expression_df.transpose()
gene_expression_df_transposed.columns = gene_expression_df_transposed.iloc[0]
gene_expression_df_transposed_index = gene_expression_df_transposed.drop("ID_REF", axis=0)
gene_expression_df_transposed_index_final = gene_expression_df_transposed_index.astype("float64")
df = pd.merge(patient_df, gene_expression_df_transposed_index_final, right_index=True, left_on='ID_REF')

In [5]:
# gene_id, gene_categorical, gene_continuous = ai.data_info(gene_expression_df, gene_expression_data, 20)
# patient_id, patient_categorical, patient_continuous = ai.data_info(patient_df, patient_data, 20)
df_id, df_cat, df_cont = ai.data_info(df, "Combined Colorectal Cancer Dataset", 20)

DATA FILE:
----------------------------------------------------------------------------------------------------
Combined Colorectal Cancer Dataset
----------------------------------------------------------------------------------------------------


DIMENSIONS:
----------------------------------------------------------------------------------------------------
Entries: 62
Features: 1944
----------------------------------------------------------------------------------------------------


CATEGORICAL FEATURES:
----------------------------------------------------------------------------------------------------
Features     Data Type    Categories & Counts
-----------  -----------  ---------------------------------------------------
Dukes Stage  object       {'A': 16, 'B': 14, 'C': 20, 'D': 12}
Gender       object       {'Female': 14, 'Male': 48}
Location     object       {'Colon': 2, 'Left': 20, 'Rectum': 18, 'Right': 22}
DFS event    float64      {0.0: 25, 1.0: 37}
Adj_Radio    float64 

### Data Preprocessing

In [6]:
# data split
X, y, X_train, y_train, X_test, y_test, X_val, y_val = ai.data_split(df, target="Dukes Stage", method="tt", random_state=0, train_proportion=0.8, test_proportion=0.2, validation_proportion=0.25, stratify="Yes")

In [7]:
# treat missing values
train_treat_na, test_treat_na = ai.treat_nan(X_train, y_train, X_test, y_test, df_id, df_cat, df_cont, target="Dukes Stage", drop_nan_rows=False, impute_cutoff=0.5, categorical_imputer="mode", continuous_imputer="median")

In [8]:
# treat duplicate values
train_df_treat_duplicate, test_df_treat_duplicate = ai.treat_duplicate(train_treat_na, test_treat_na, keep_in="first")

In [9]:
# treat outliers
train_df_treat_outliers, test_df_treat_outliers = ai.treat_outliers(train_df_treat_duplicate, test_df_treat_duplicate, df_id, df_cat, df_cont, target="Dukes Stage", method="svm", outlier_fraction=0.35)

### Data Exploration

In [12]:
ai.custom_boxplot(train_df_treat_outliers, x_var="Dukes Stage", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Gene", ytitle="Gene Expression Value", dropdown_desrciption="Select: ", stratify="No", group="No")

In [13]:
ai.custom_boxplot(train_df_treat_outliers, x_var="Dukes Stage", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Gene", ytitle="Gene Expression Value", dropdown_desrciption="Select: ", stratify="No", group="Yes")

In [17]:
ai.custom_boxplot(train_df_treat_outliers, x_var="Dukes Stage", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Dukes Stage", ytitle="Gene Expression Value", dropdown_desrciption="Select: ", stratify="Yes", group="No")

In [16]:
ai.custom_boxplot(train_df_treat_outliers, x_var="Dukes Stage", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Dukes Stage", ytitle="Gene Expression Value", dropdown_desrciption="Select: ", stratify="Yes", group="Yes")

In [18]:
ai.custom_boxplot(train_df_treat_outliers, x_var="Location", y_var=list(df.columns[9:]), group_var="Gender", title="Gene Expression of: ", xtitle="Location", ytitle="Gene Expression Value", dropdown_desrciption="Select: ", stratify="Yes", group="Yes")

#### Experimental code

In [16]:
selection_list = []
for gene in df.columns[9:]:
    selection_list.append((gene, gene))

In [27]:
import ipywidgets as widgets
import plotly.express as px
from plotly.graph_objs import graph_objs as go

def boxplot_creator(selection):
    fig = go.Figure()
    fig.add_trace(go.Box(x=train_df_treat_outliers['Dukes Stage'], y=train_df_treat_outliers[selection]))
    fig.update_layout(legend_title_text = "Gender")
    # fig = px.box(train_df_treat_outliers, x='Dukes Stage', y=selection, color="Gender", title=f"Gene: {selection}", labels={selection:selection})
    fig.show()
widgets.interactive(boxplot_creator, selection=widgets.Dropdown(options=selection_list, description='Select Gene:'));

In [10]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [11]:
df.head()

ID_REF  Age (in years) Dukes Stage  Gender Location  DFS (in months)  \
0  GSM877126            62.0           A    Male     Left            108.0   
1  GSM877127            77.0           B    Male     Left             40.0   
2  GSM877128            66.0           C  Female     Left             49.0   
3  GSM877129            72.0           D  Female     Left             45.0   
4  GSM877130            75.0           C    Male     Left             40.0   

   DFS event  Adj_Radio  Adj_Chem    117_at  1007_s_at   1053_at    121_at  \
0        0.0        1.0       0.0  6.029834   9.468893  7.187090  6.807582   
1        1.0        1.0       0.0  6.344321   9.994204  7.705626  6.893130   
2        0.0        1.0       0.0  5.023127  10.409823  7.934057  7.165602   
3        0.0        1.0       1.0  4.612727  10.596921  7.446374  7.071417   
4        0.0        0.0       1.0  5.865944  10.252547  7.141105  7.103383   

   1255_g_at   1294_at   1316_at   1320_at  1405_i_at   1431_at   1438_at  \
0   3.084591  7.397013  6.525426  3.848253   7.572331  3.748200  6.957488   
1   3.118704  7.163797  6.391195  4.202085   8.295322  3.114340  7.366494   
2   2.984632  7.375224  5.538846  4.074462   7.055579  4.172270  9.190242   
3   2.912851  7.596233  5.989148  3.738205   9.683806  3.625413  7.493741   
4   3.383726  7.278244  5.678236  4.141955   7.360892  3.338311  6.068710   

    1487_at  1494_f_at  1552256_a_at  1552257_a_at  1552258_at  1552261_at  \
0  8.057704   4.311960      7.246198      8.998012    3.912720    4.206709   
1  8.450329   4.257666      8.113613      9.154566    4.223799    3.359484   
2  8.186155   4.390612      7.038909      8.273925    3.988332    3.434758   
3  7.398235   4.308245      7.547753      9.075829    3.885780    4.225294   
4  7.919004   4.188543      7.921648      7.992741    4.960633    3.391169   

   1552263_at  1552264_a_at  1552266_at  1552269_at  1552271_at  1552272_a_at  \
0    6.293766      6.585635    2.837637    3.024119    4.561616      4.371035   
1    7.781680      8.131629    2.408522    2.942956    3.882600      4.264255   
2    5.245886      6.081574    2.800716    2.845071    4.749350      5.269892   
3    5.958728      6.779613    2.659192    3.280929    4.333246      4.509829   
4    6.249177      6.517431    2.556627    2.872373    4.438411      3.997675   

   1552274_at  1552275_s_at  1552276_a_at  1552277_a_at  1552278_a_at  \
0    6.120874      4.361976      4.765517      7.101751      3.900317   
1    6.112272      4.974505      4.482178      7.397170      4.620965   
2    4.948306      4.151721      4.577981      7.800087      4.231365   
3    6.325548      4.535780      4.417799      7.025963      4.167651   
4    6.070412      5.086089      4.243861      7.723245      4.286986   

   1552279_a_at  1552280_at  1552281_at  1552283_s_at  1552286_at  \
0      5.035382    2.830732    6.470622      3.906776    5.081563   
1      5.417600    3.424860    7.581700      4.590770    4.608948   
2      5.261286    3.240356    6.405948      5.156768    5.788224   
3      5.284253    3.010893    6.372876      6.108916    5.102382   
4      5.683226    3.116138    7.471077      3.897945    4.986585   

   1552287_s_at  1552288_at  1552289_a_at  1552291_at  1552293_at  \
0      6.837268    3.297914      4.153212    6.358204    3.360942   
1      6.190298    3.921998      5.008631    6.512021    3.815529   
2      6.788046    3.498495      3.475740    6.277524    3.473501   
3      6.382470    3.253702      4.094617    6.637175    3.831071   
4      6.793243    3.222510      3.844001    5.937285    3.699799   

   1552295_a_at  1552296_at  1552299_at  1552301_a_at  1552302_at  \
0      5.999121    3.611584    4.041372      3.673284    4.436078   
1      5.832778    3.944787    4.022736      3.668590    3.242581   
2      6.072805    3.626742    5.108468      3.723708    3.474229   
3      6.029997    4.252454    4.464719      3.599279    3.869152   
4      6.698944    3.816713    4.55

In [12]:
# month = widgets.IntSlider(
#     value=1.0,
#     min=1.0,
#     max=12.0,
#     step=1.0,
#     description='Month:',
#     continuous_update=False
# )

# use_date = widgets.Checkbox(
#     description='Date: ',
#     value=True,
# )

# container = widgets.HBox(children=[use_date, month])

# textbox = widgets.Dropdown(
#     description='Airline:   ',
#     value='DL',
#     options=df['carrier'].unique().tolist()
# )

# origin = widgets.Dropdown(
#     options=list(df['origin'].unique()),
#     value='LGA',
#     description='Origin Airport:',
# )


# # Assign an empty figure widget with two traces
# trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
# trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
# g = go.FigureWidget(data=[trace1, trace2],
#                     layout=go.Layout(
                    #     title=dict(
                    #         text='NYC FlightDatabase'
                    #     ),
                    #     barmode='overlay'
                    # ))

In [11]:
gene_list = []
for gene in df.columns[9:]:
    gene_list.append(gene)

In [12]:
# gender = widgets.Dropdown(
#     description='Gender: ',
#     value='Male',
#     options=df['Gender'].unique().tolist()
# )

# location = widgets.Dropdown(
#     options=list(df['Location'].unique()),
#     value='Left',
#     description='Tumor Location:',
# )

gene = widgets.Dropdown(
    description='Gene: ',
    value='117_at',
    options=gene_list
)

# Assign an empty figure widget with two traces
trace1 = go.Box(x=df["Dukes Stage"], y=df[gene.value], boxmean=True)
# trace1 = go.Histogram(x=df[gene.value], opacity=0.75, name=gene.value)
# trace2 = go.Histogram(x=df[gene.value], opacity=0.75, name=gene.value)
g = go.FigureWidget(data=[trace1], layout=go.Layout(title=dict(text=f'Gene Expression of 117_at'), barmode='overlay'))

In [13]:
def validate():
    if gene.value in gene_list: #and location.value in df['Location'].unique() and gene.value in df.columns[9:].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if gene.value:
            temp_df = df[gene.value]
            # filter_list = [i for i in zip(df["Gender"] == gender.value)]
            # temp_df = df[filter_list]
            # temp_df = temp_df[gene.value]

        else:
            temp_df = df["117_at"]
            # filter_list = [i for i in zip(df["Gender"] == 'Male')]
            # temp_df = df[filter_list]
            # temp_df = temp_df["117_at"]
        
        x1 = temp_df
        # x2 = temp_df[gene.value]
        with g.batch_update():
            g.data[0].y = x1
            # g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Dukes Stages'
            g.layout.yaxis.title = 'Gene Expression'
            g.layout.title = f"Gene Expression of {gene.value}"


# location.observe(response, names="value")
gene.observe(response, names="value")
# gender.observe(response, names="value")

In [14]:
container = widgets.VBox([gene])
widgets.VBox([container, g])